<a href="https://colab.research.google.com/github/onism/MyLearning/blob/master/TransferLearningTricks_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torchvision import models

transfer_model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


Next, we need to freeze the layers. 

In [3]:
for name, params in transfer_model.named_parameters():
    print(name)
    params.requires_grad=False

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

**TIP**

We might not want to freeze the BatchNorm layers in a model, as they will be trained to approximate the mean and standard deviation of the dataset that the model was originally trained on.

In [4]:
for name, param in transfer_model.named_parameters():
    if 'bn' not in name:
        param.requires_grad = False 
    else:
        param.requires_grad = True 

Then we need to replace the final classification block with a new one.



In [5]:
import torch.nn as nn 
transfer_model.fc = nn.Sequential(
    nn.Linear( transfer_model.fc.in_features, 500 ),
    nn.ReLU(),
    nn.Linear(500,2)
)

**Fine tuning**

In [10]:
from torch import optim

optimizer = optim.Adam(  
    [
     {'params': transfer_model.layer4.parameters(), 'lr':1e-4},
     {'params': transfer_model.layer3.parameters(), 'lr':1e-4},
    ],
    lr = 1e-3
)

In [11]:
unfreeze_layers = [transfer_model.layer3, transfer_model.layer4]

for layer in unfreeze_layers:
    for param in layer.parameters():
        param.requires_grad = True 